In [40]:
#ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline
from numpy import nan

#データの読み込み
df = pd.read_csv('train_taitanic.csv')

In [41]:
#欠損値処理

#敬称による分類、繰り返し文{'Miss.':0,'Mrs.':1,'Master':2,'Mr':3}
#Msは1人なのでMrsに追加
#3には数人Dr.などいるがMr.として考える

list_n = []
for name in df.loc[:,['Name']].values.reshape(-1):
    if 'Miss.' in name:
        list_n.append(0)
    elif 'Mrs.' in name or 'Ms.' in name:
        list_n.append(1)
    elif 'Master.' in name:
        list_n.append(2)
    else:
        list_n.append(3)

In [42]:
#dfにHonorificカラムを追加
df['Honorific'] = np.array(list_n)

In [43]:
#Ageの欠損値を敬称ごとの平均値で補完

#Ageが欠損しているindexを取り出しのlistを作成
age_nan = df.Age.isnull()
list_nan = age_nan.values
list_t = []
for i in range(len(list_nan)):
    if list_nan[i] == True:
        list_t.append(i)
        
#list_tに対して敬称ごとの年齢の中央値が対応するlistの作成
list_m = []
age_mean = df.groupby('Honorific').Age.mean()
for i in list_t:
    list_m.append(age_mean[df.Honorific[i]])
    
#df_ageカラムを作成し、dfに追加
df_age = df.loc[:,['Age']]
for j,k in zip(list_t, list_m):    
    df_age.Age[j] = k

df['Age_mod'] = df_age

In [44]:
#Sexをダミー変数化
df['Female'] = df['Sex'].map({'male':0,'female':1})

#Pclassをダミー変数化
#Embarkedをダミー変数化
pclass = pd.get_dummies(df['Pclass'],prefix ='Class')
embarked = pd.get_dummies(df['Embarked'], prefix='Embarked')

#df,pclass,embarkedを結合
df = pd.concat([df, pclass, embarked], axis=1)

In [45]:
#説明変数と目的変数を指定
X_1 = df.loc[:,['Class_1','Female','Age_mod',
                'Honorific','Fare','SibSp']].values
y_1 = df.loc[:,['Survived']].values.reshape(-1)

In [46]:
# 勾配ブースティング木で学習させる
from sklearn.ensemble import GradientBoostingClassifier
gbct_1 = GradientBoostingClassifier(n_estimators=10,random_state=1, max_depth=4, learning_rate=0.1)
gbct_1.fit(X_1, y_1)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=4,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10,
              presort='auto', random_state=1, subsample=1.0, verbose=0,
              warm_start=False)

In [47]:
df_test = pd.read_csv('test_taitanic.csv')

In [48]:
#欠損値処理

#敬称による分類、繰り返し文{'Miss.':0,'Mrs.':1,'Master':2,'Mr':3}
#Msは1人なのでMrsに追加
#3には数人Dr.などいるがMr.として考える

list_n = []
for name in df_test.loc[:,['Name']].values.reshape(-1):
    if 'Miss.' in name:
        list_n.append(0)
    elif 'Mrs.' in name or 'Ms.' in name:
        list_n.append(1)
    elif 'Master.' in name:
        list_n.append(2)
    else:
        list_n.append(3)

In [49]:
#df_testにHonorificカラムを追加
df_test['Honorific'] = np.array(list_n)

In [50]:
#Ageの欠損値を敬称ごとの平均値で補完

#Ageが欠損しているindexを取り出しのlistを作成
age_nan = df_test.Age.isnull()
list_nan = age_nan.values
list_t = []
for i in range(len(list_nan)):
    if list_nan[i] == True:
        list_t.append(i)
        
#list_tに対して敬称ごとの年齢の中央値が対応するlistの作成
list_m = []
age_mean = df_test.groupby('Honorific').Age.mean()
for i in list_t:
    list_m.append(age_mean[df_test.Honorific[i]])
    
#df_ageカラムを作成し、dfに追加
df_age = df_test.loc[:,['Age']]
for j,k in zip(list_t, list_m):    
    df_age.Age[j] = k

df_test['Age_mod'] = df_age

In [51]:
#Sexをダミー変数化
df_test['Female'] = df_test['Sex'].map({'male':0,'female':1})

#Pclassをダミー変数化
#Embarkedをダミー変数化
pclass = pd.get_dummies(df_test['Pclass'],prefix ='Class')
embarked = pd.get_dummies(df_test['Embarked'], prefix='Embarked')

#df,pclass,embarkedを結合
df_test = pd.concat([df_test, pclass, embarked], axis=1)

In [57]:
#説明変数を指定
X_fin = df_test.loc[:,['Class_1','Female','Age_mod',
                'Honorific','Fare','SibSp']]
X_fin.Fare[152] = 10

/Users/shun/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [59]:
result = gbct_1.predict(X_fin)
result_ = pd.DataFrame(result, columns=['Survived'])
index_list = list(range(892,1310,1))
result_.index = index_list
result_.to_csv('result.csv')